In [1]:
# !pip install transformers datasets accelerate peft bitsandbytes

In [2]:
import gc
import torch
import numpy as np

from datasets import Dataset
from torch.utils.data import DataLoader

from transformers import AdamW
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
def print_gpu_utilization():
    if torch.cuda.is_available():
        used_memory = torch.cuda.memory_allocated() / (1024 ** 3)
        print(f"GPU 메모리 사용량 : {used_memory:.4f}GB")

    else:
        print("GPU 환경이 아닙니다.")

print_gpu_utilization()

GPU 메모리 사용량 : 0.0000GB


In [4]:
def load_model_and_tokenizer(model_id, peft=None):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"": 0})

    print_gpu_utilization()
    return model, tokenizer

model_id = "EleutherAI/polyglot-ko-1.3b"
model, tokenizer = load_model_and_tokenizer(model_id)

# 모델의 파라미터 데이터 타입 출력
first_param = next(model.parameters())
print("모델 파라미터 데이터 타입 : ", first_param.dtype)


/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량 : 2.5989GB
모델 파라미터 데이터 타입 :  torch.float16


In [5]:
def estimate_memory_of_gradients(model):
    total_memory = 0  # 총 메모리 저장 변수 초기화
    for param in model.parameters():  # 모델의 모든 파라미터에 대해 반복
        if param.grad is not None:  # 파라미터에 그레디언트가 있을 때만 계산
            # 각 그레디언트 텐서의 요소 개수와 요소 크기(바이트) 곱을 추가
            total_memory += param.grad.nelement() * param.grad.element_size()
    return total_memory  # 총 메모리 반환

def estimate_memory_of_optimizer(optimizer):
    total_memory = 0  # 총 메모리 저장 변수 초기화
    for state in optimizer.state.values():  # 옵티마이저의 모든 상태에 대해 반복
        for k, v in state.items():  # 각 상태의 키와 값을 반복
            if torch.is_tensor(v):  # 값이 텐서인지 확인
                # 각 텐서의 요소 개수와 요소 크기(바이트) 곱을 추가
                total_memory += v.nelement() * v.element_size()
    return total_memory  # 총 메모리 반환

In [6]:
def train_model(model, dataset, training_args):
    if training_args.gradient_checkpointing:
        model.gradient_checkpointing_enable()

    train_dataloader = DataLoader(dataset, batch_size=training_args.per_device_train_batch_size)
    optimizer = AdamW(model.parameters())
    
    model.train()
    gpu_utilization_printed = False
    for step, batch in enumerate(train_dataloader, start=1):
        batch = {k : v.to(model.device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        loss = loss / training_args.gradient_accumulation_steps
        loss.backward()

        if step % training_args.gradient_accumulation_steps == 0:
            optimizer.step()
            gradients_memory = estimate_memory_of_gradients(model)
            optimizer_memory = estimate_memory_of_optimizer(optimizer)

            if not gpu_utilization_printed:
                print_gpu_utilization()
                gpu_utilization_printed = True

            optimizer.zero_grad()

    print(f"옵티마이저 상태의 메모리 사용량 : {optimizer_memory / (1024 ** 3):.4f}GB")
    print(f"그레디언트 메모리 사용량 : {gradients_memory / (1024 ** 3):.4f}GB")

In [7]:
def make_dummy_dataset():
    seq_len, dataset_size = 256, 64
    dummy_data = {
        "input_ids" : np.random.randint(100, 30000, (dataset_size, seq_len)),
        "labels" : np.random.randint(100, 30000, (dataset_size, seq_len))
    }

    dataset = Dataset.from_dict(dummy_data)
    dataset.set_format("pt")

    return dataset

In [8]:
def cleanup():
    if 'model' in globals():
        del globals()['model']

    if 'dataset' in globals():
        del globals()['dataset']

    gc.collect()
    torch.cuda.empty_cache()

In [9]:
def gpu_memory_experiment(batch_size,
                          gradient_accumulation_steps=1,
                          gradient_checkpointing=False,
                          model_id="EleutherAI/polyglot-ko-1.3b",
                          peft=None):
    
    print(f"배치 크기 : {batch_size}")
    model, tokenizer = load_model_and_tokenizer(model_id, peft=peft)

    if gradient_checkpointing == True or peft == 'qlora':
        model.config.use_cache = False

    dataset = make_dummy_dataset()

    training_args = TrainingArguments(
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        gradient_checkpointing=gradient_checkpointing,
        output_dir="./train_results",
        num_train_epochs=1
    )

    try:
        train_model(model, dataset, training_args)
    except RuntimeError as e:
        if "CUDA out of memory" in str(e):
            print(e)
        else:
            raise e
    finally:
        del model, dataset
        gc.collect()
        torch.cuda.empty_cache()
        print_gpu_utilization()

In [11]:
cleanup()
print_gpu_utilization()

for batch_size in [4, 8, 16]:
    gpu_memory_experiment(batch_size)

    torch.cuda.empty_cache()

GPU 메모리 사용량 : 0.1311GB
배치 크기 : 4


/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량 : 2.7300GB
GPU 메모리 사용량 : 10.7011GB


/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량 : 2.7300GB


/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


GPU 메모리 사용량 : 12.2795GB
옵티마이저 상태의 메모리 사용량 : 4.9614GB
그레디언트 메모리 사용량 : 2.4807GB
GPU 메모리 사용량 : 0.1311GB
